In [1]:
import pyarrow # must occur prior to ray import
import ray
from ray import tune
from ray.tune import ExperimentAnalysis
from ray.tune.search.hyperopt import HyperOptSearch
import datetime
import numpy as np
import pandas as pd
import random
import seaborn as sns; sns.set()
from collections import Counter
from datasets import load_from_disk
from scipy.stats import ranksums
from sklearn.metrics import accuracy_score, f1_score
from transformers import BertForSequenceClassification
from transformers import Trainer
from transformers.training_args import TrainingArguments

from geneformer import DataCollatorForCellClassification
from geneformer import TranscriptomeTokenizer
import anndata as ad
import pandas as pd

In [2]:
file_path = "/home/wangxihe/AF_atlas/Data/单细胞分析/HF心房心肌细胞/Atrial_ensembl.h5ad"
# 使用 read_h5ad 函数读取文件
HLCA = ad.read_h5ad(file_path)

In [3]:
HLCA

AnnData object with n_obs × n_vars = 6289 × 31665
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'ID', 'Barcode', 'Type', 'Individual', 'Age', 'Gender', 'Dispense.Order', 'X384.Well.Plate.Location', 'Chip.Row.ID', 'Chip.Column.ID', 'Image.ID', 'Barcode.Read.Pairs', 'Distinct.UMIs', 'ERCC.Read.Pairs', 'Trimmed.Read.Pairs', 'NoContam.Read.Pairs', 'Mitochondria.Alignments', 'Mitochondria.Read.Pairs', 'Total.Barcode.Alignments', 'Distinct.Genes.w..Alignments', 'Distinct.Gene.UMI.Combos', 'Aligned', 'Assigned', 'Ambiguity', 'Chimera', 'Duplicate', 'FragementLength', 'MappingQuality', 'MultiMapping', 'NoFeatures', 'Nonjunction', 'Secondary', 'Unmapped', 'mito.perc', 'CellType', 'percent.mt', 'nCount_SCT', 'nFeature_SCT', 'SCT_snn_res.0.5', 'seurat_clusters', 'cell_type', 'cell_type_own', 'condition'
    var: 'features'

In [4]:
HLCA.var.features

ENSG00000000003    ENSG00000000003
ENSG00000000419    ENSG00000000419
ENSG00000000457    ENSG00000000457
ENSG00000000938    ENSG00000000938
ENSG00000000971    ENSG00000000971
                        ...       
ENSG00000282511    ENSG00000282511
ENSG00000282520    ENSG00000282520
ENSG00000282591    ENSG00000282591
ENSG00000282599    ENSG00000282599
ENSG00000282625    ENSG00000282625
Name: features, Length: 31665, dtype: object

In [5]:
total_read_counts = HLCA.X.sum(axis=1)
HLCA.obs['n_counts'] = total_read_counts
HLCA.var['ensembl_id'] = HLCA.var.index

In [6]:
condition_counts = HLCA.obs['condition'].value_counts()
cell_type_counts = HLCA.obs['cell_type'].value_counts()

In [7]:
condition_counts

condition
normal    4108
HF        2181
Name: count, dtype: int64

In [9]:
cell_type_counts # 1 CM;  0 EC; 2 FB; 3 ACTA2+FB; 8: 8; 5 SMC; 6 MP; 7:12  ;   8 Mesothelial

cell_type
1    2746
0    1416
2     630
3     440
4     363
5     261
6     185
7     137
8     111
Name: count, dtype: int64

In [10]:
obs_df = HLCA.obs
CM_obs = obs_df.loc[obs_df['cell_type'] == 1]
CM= HLCA[CM_obs.index]

In [11]:
CM

View of AnnData object with n_obs × n_vars = 2746 × 31665
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'ID', 'Barcode', 'Type', 'Individual', 'Age', 'Gender', 'Dispense.Order', 'X384.Well.Plate.Location', 'Chip.Row.ID', 'Chip.Column.ID', 'Image.ID', 'Barcode.Read.Pairs', 'Distinct.UMIs', 'ERCC.Read.Pairs', 'Trimmed.Read.Pairs', 'NoContam.Read.Pairs', 'Mitochondria.Alignments', 'Mitochondria.Read.Pairs', 'Total.Barcode.Alignments', 'Distinct.Genes.w..Alignments', 'Distinct.Gene.UMI.Combos', 'Aligned', 'Assigned', 'Ambiguity', 'Chimera', 'Duplicate', 'FragementLength', 'MappingQuality', 'MultiMapping', 'NoFeatures', 'Nonjunction', 'Secondary', 'Unmapped', 'mito.perc', 'CellType', 'percent.mt', 'nCount_SCT', 'nFeature_SCT', 'SCT_snn_res.0.5', 'seurat_clusters', 'cell_type', 'cell_type_own', 'condition', 'n_counts'
    var: 'features', 'ensembl_id'

In [13]:
CM.__dict__['_raw'].__dict__['_var'] = CM.__dict__['_raw'].__dict__['_var'].rename(columns={'_index': 'features'})
save_path = "/home/wangxihe/AF_atlas/Data/单细胞分析/HF心房心肌细胞/CM.h5ad"
# 使用 write_h5ad 方法保存 AnnData 对象
CM.write_h5ad(save_path)

In [14]:
tk = TranscriptomeTokenizer({"cell_type": "cell_type_own","condition": "condition"}, nproc=1)

In [17]:
tk.tokenize_data("/home/wangxihe/AF_atlas/Data/单细胞分析/HF心房心肌细胞/CM", 
                 "/home/wangxihe/AF_atlas/Data/单细胞分析/HF心房心肌细胞/CM/", 
                 "CM",
                file_format="h5ad")

Tokenizing /home/wangxihe/AF_atlas/Data/单细胞分析/HF心房心肌细胞/CM/CM.h5ad
/home/wangxihe/AF_atlas/Data/单细胞分析/HF心房心肌细胞/CM/CM.h5ad has no column attribute 'filter_pass'; tokenizing all cells.
Creating dataset.


Map:   0%|          | 0/2746 [00:00<?, ? examples/s]

Map:   0%|          | 0/2746 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/2746 [00:00<?, ? examples/s]